## Imports

In [29]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
import tabula

## Data read-in

In [30]:
df = pd.read_csv('Class A Office DTLA - Sheet1.csv')

## Map prep

In [31]:
# Create full address for geocoding
df['full_address'] = df['Address'] + ' ' + df['DTLA area'] + ' Los Angeles' 

In [32]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [33]:
# Remove NaNs
df = df.dropna(subset='Property name')

In [34]:
df.at[19,'Address']=('505-555 South Flower Street')

In [35]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

df['geocoded'] = df['full_address'].apply(geocode)

In [36]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].str.strip('()').str.split(', ', expand=True)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

In [37]:
df.columns

Index(['Property name', 'Address', 'Owner', 'Area', 'Stories', 'Built',
       'Previous valuation', '2022 valuation', 'DTLA area', 'Unnamed: 9',
       'Unnamed: 10', 'Unnamed: 11', 'full_address', 'geocoded', 'lat', 'lon'],
      dtype='object')

In [38]:
def popup_html(row):
    building_name = row['Property name']
    address = row['Address']
    owner = row['Owner']
    area = row['Area']

    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(building_name) + '''<br>
    <strong>Address: </strong>{}'''.format(address) + '''<br>
    <strong>Owner: </strong>{}'''.format(owner) + '''<br>
    <strong>Square Footage: </strong>{}'''.format(area) + '''<br>
    </html>
    '''
    return html

In [39]:
# df.at[19,'lat']=(34.0523403990926)
# df.at[19,'lon']=(-118.25783008004237)

In [40]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=10)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'LA Class A Office Space')

### Create map container ###
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=9.5,tiles=None)

for index, row in df.iterrows():
    marker = folium.Marker(
        location=[row['lat'], row['lon']],
        radius=5,
        fill=True,
        popup=folium.Popup(popup_html(row), max_width=400))
    marker.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
            
# Display map
m

In [41]:
m.save('index.html')

## Map URL Snagger

In [42]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/ClassA_office_DTLA
